In [6]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('..')
from eegdash.signalstore_data_utils import SignalstoreBIDS
from eegdash.signalstore_data_utils import SignalstoreOpenneuro
import s3fs
import os
import tempfile
import mne
import h5py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


test signalstore speed

In [7]:
eegdash = SignalstoreOpenneuro(
    is_public=False,
    local_filesystem=False,
)
eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Pinged your deployment. You successfully connected to MongoDB!
Found 1 records


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.


KeyboardInterrupt: 

In [8]:
%%time
eegdash = SignalstoreBIDS(
    is_public=True,
    local_filesystem=False,
)
eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Pinged your deployment. You successfully connected to MongoDB!


OSError: [Errno 22] The provided token is malformed or otherwise invalid.

In [3]:
data = eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Found 1 records


In [10]:
data[0].to_numpy()

array([[3.90708122e-05, 4.42323761e-05, 3.64698410e-05, ...,
        4.36852455e-05, 4.51106300e-05, 4.22494888e-05],
       [4.68087311e-05, 4.57453194e-05, 4.40404015e-05, ...,
        2.10300961e-05, 2.51476688e-05, 2.52572155e-05],
       [3.52780228e-05, 3.70137100e-05, 4.12147102e-05, ...,
        5.17162743e-05, 5.74016876e-05, 5.65963058e-05],
       ...,
       [7.02251358e-05, 7.77657928e-05, 6.62534866e-05, ...,
        6.97815170e-05, 1.01796478e-04, 9.74829407e-05],
       [6.03754272e-05, 2.69368916e-05, 3.46484642e-05, ...,
        6.66851044e-05, 6.78498688e-05, 6.35415993e-05],
       [3.64559059e-05, 4.87539520e-05, 4.18273659e-05, ...,
        4.89408951e-05, 6.06918449e-05, 5.85218697e-05]],
      shape=(74, 747750))

Compare reading .set vs .nwb files from S3

In [8]:
filesystem = s3fs.S3FileSystem(anon=True, client_kwargs={'region_name': 'us-east-2'})

In [6]:
def load_eeg_data_from_bids_file(bids_file_path, data_format):
    if data_format == 'set':
        eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)
    elif data_format == 'nwb':
        with pynwb.NWBHDF5IO(bids_file_path, "r") as io:
            eeg_data = io.read()
    return eeg_data

def load_eeg_data_from_s3(s3path, data_format):
    with tempfile.NamedTemporaryFile(delete=False, suffix='.set') as tmp:
        with filesystem.open(s3path) as s3_file:
            tmp.write(s3_file.read())
        tmp_path = tmp.name
        eeg_data = load_eeg_data_from_bids_file(tmp_path, data_format)
        os.unlink(tmp_path)
        return eeg_data

In [9]:
%%time
load_eeg_data_from_s3('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.set', 'set')

/tmp/ipykernel_2262761/2232965347.py:3: RuntimeWarning: Unknown types found, setting as type EEG:
ekg: ['EEG063', 'EEG064']
heog: ['EEG061']
veog: ['EEG062']
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)
/tmp/ipykernel_2262761/2232965347.py:3: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)


CPU times: user 1.99 s, sys: 1.1 s, total: 3.09 s
Wall time: 20.1 s


/tmp/ipykernel_2262761/2232965347.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)


<RawEEGLAB | tmpxan14ibh.set, 74 x 747750 (2991.0 s), ~422.3 MiB, data loaded>

In [18]:
%%time
load_eeg_data_from_s3('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.nwb', 'nwb')

/home/dung/Documents/EEG-Dash-Data/.venv/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/dung/Documents/EEG-Dash-Data/.venv/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/dung/Documents/EEG-Dash-Data/.venv/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


ConstructError: (root/general/extracellular_ephys/all_electrodes GroupBuilder {'attributes': {'description': 'all electrodes', 'location': array([], dtype=object), 'namespace': 'core', 'neurodata_type': 'ElectrodeGroup', 'object_id': '28a6d082-14df-4e89-bd22-226dfb046e62'}, 'groups': {}, 'datasets': {}, 'links': {'device': root/general/extracellular_ephys/all_electrodes/device LinkBuilder {'builder': root/general/devices/array GroupBuilder {'attributes': {'namespace': 'core', 'neurodata_type': 'Device', 'object_id': '0612f0a8-8ca0-41a9-86a4-b32a3b3f107e'}, 'groups': {}, 'datasets': {}, 'links': {}}}}}, "Could not construct ElectrodeGroup object due to: ElectrodeGroup.__init__: incorrect type for 'location' (got 'ndarray', expected 'str')")

In [26]:
%%time
with filesystem.open('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.nwb', "rb") as f:
    with h5py.File(f) as file:
        with pynwb.NWBHDF5IO(file=file) as io:
            nwbfile = io.read()
            print(nwbfile)

ConstructError: (root/general/extracellular_ephys/all_electrodes GroupBuilder {'attributes': {'description': 'all electrodes', 'location': array([], dtype=object), 'namespace': 'core', 'neurodata_type': 'ElectrodeGroup', 'object_id': '2a87fe4b-0cc3-458d-988b-34b55a2826d9'}, 'groups': {}, 'datasets': {}, 'links': {'device': root/general/extracellular_ephys/all_electrodes/device LinkBuilder {'builder': root/general/devices/array GroupBuilder {'attributes': {'namespace': 'core', 'neurodata_type': 'Device', 'object_id': '5f38dc95-4ad3-4700-a70a-58f803b99c82'}, 'groups': {}, 'datasets': {}, 'links': {}}}}}, "Could not construct ElectrodeGroup object due to: ElectrodeGroup.__init__: incorrect type for 'location' (got 'ndarray', expected 'str')")